## Projeto 02 - Como pegar dados de um site com Python? Pegando ETFs do mundo inteiro.

### Desafio:

- Construir um código que vá no site etf.com e busque dados de todos os etfs do mercado americano e conquequentemente do mundo. Rentabilidades, patrimonio, gestora, taxa...


### Escolher o navegador

In [43]:
!pip install webdriver-manager

In [44]:
!pip install selenium


In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [46]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

driver.get('https://www.etf.com/etfanalytics/etf-finder')

### Achar todos os elementos necessários dentro do HTML do site Expandindo a tabela para 100 itens.



In [50]:
time.sleep(5)

botao_100 = driver.find_element('xpath',
                               '/html/body/div[4]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span')

driver.execute_script('arguments[0].click();', botao_100)

#outra opção de código para clicar caso o .click não funcione
#dirver.execute_script('arguments[0].click();', botao_100)

### Achar todos os elementos necessários dentro do HTML  do site - Pegando o número de páginas da tabela.

In [51]:
numero_paginas = driver.find_element('xpath', '//*[@id="totalPages"]')
numero_paginas = numero_paginas.text.replace('of ','')
numero_paginas = int(numero_paginas)

numero_paginas

32

### Lendo a tabela de dados - Lendo a tabela de dados básicos.

In [52]:
elemento = driver.find_element('xpath','//*[@id="finderTable"]')

  
html_tabela = elemento.get_attribute('outerHTML')
    
tabela = pd.read_html(str(html_tabela))[0]

tabela

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$371.98B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$305.66B
2,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.90B
3,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$281.86B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.39B
...,...,...,...,...,...,...
95,BIV,Vanguard Intermediate-Term Bond ETF,"Fixed Income: U.S. - Broad Market, Broad-based...",Vanguard,0.04%,$13.98B
96,XLI,Industrial Select Sector SPDR Fund,Equity: U.S. Industrials,State Street Global Advisors,0.10%,$13.89B
97,GDX,VanEck Gold Miners ETF,Equity: Global Gold Miners,VanEck,0.51%,$13.85B
98,TQQQ,ProShares UltraPro QQQ,Leveraged Equity: U.S. - Large Cap,ProShares,0.86%,$13.75B


In [53]:
lista_tabela_por_pagina =[]

elemento = driver.find_element('xpath','//*[@id="finderTable"]')

for pagina in range(1, numero_paginas +1):
  
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
   
    botao_avancar_pagina = driver.find_element('xpath','//*[@id="nextPage"]')

    botao_avancar_pagina.click()
    
tabela_cadastro_etfs = pd.concat(lista_tabela_por_pagina)

tabela_cadastro_etfs

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$371.98B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$305.66B
2,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.90B
3,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$281.86B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.39B
...,...,...,...,...,...,...
18,CVLC,Calvert US Large-Cap Core Responsible Index ETF,Equity: U.S. - Large Cap,Morgan Stanley,0.15%,--
19,CVMC,Calvert US Mid-Cap Core Responsible Index ETF,Equity: U.S. - Mid Cap,Morgan Stanley,0.15%,--
20,CVIE,Calvert International Responsible Index ETF,Equity: Developed Markets Ex-U.S. - Large Cap,Morgan Stanley,0.18%,--
21,CDEI,"Calvert US Large-Cap Diversity, Equity and Inc...",Equity: U.S. - Large Cap,Morgan Stanley,0.14%,--


### Ler a tabela de dados - Preenchendo um campo no site para voltar as páginas

In [54]:
formulario_de_voltar_pagina = driver.find_element('xpath', ' //*[@id="goToPage"]')

formulario_de_voltar_pagina.clear()
formulario_de_voltar_pagina.send_keys('1')
formulario_de_voltar_pagina.send_keys(u'\ue007')

### Ler a tabela de dados - Lendo a tabela de dados de rentabilidade

In [56]:
botao_mudar_pra_performance = driver.find_element('xpath','/html/body/div[4]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/ul/li[2]/span')

driver.execute_script('arguments[0].click();', botao_mudar_pra_performance)


# a partir daqui é tudo igual 

lista_tabela_por_pagina =[]

elemento = driver.find_element('xpath','//*[@id="finderTable"]')

for pagina in range(1, numero_paginas +1):
  
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
   
    botao_avancar_pagina = driver.find_element('xpath','//*[@id="nextPage"]')

    driver.execute_script('arguments[0].click();', botao_avancar_pagina)
    
tabela_rentabilidade_etfs = pd.concat(lista_tabela_por_pagina)



In [57]:
tabela_rentabilidade_etfs

,Ticker,Name,1 Month,3 Month,YTD,1 Year,3 Years,5 Years,10 Years,As Of Date
0,SPY,SPDR S&P 500 ETF Trust,2.07%,8.33%,7.87%,-7.72%,20.17%,11.41%,12.29%,04/03/23
1,IVV,iShares Core S&P 500 ETF,2.01%,8.30%,7.82%,-7.75%,20.20%,11.44%,12.34%,04/03/23
2,VOO,Vanguard S&P 500 ETF,2.11%,8.32%,7.90%,-7.73%,20.23%,11.45%,12.35%,04/03/23
3,VTI,Vanguard Total Stock Market ETF,0.93%,7.91%,7.46%,-8.92%,20.23%,10.64%,11.86%,04/03/23
4,QQQ,Invesco QQQ Trust,7.00%,21.24%,20.42%,-10.85%,21.20%,16.09%,17.77%,04/03/23
...,...,...,...,...,...,...,...,...,...,...
18,CVLC,Calvert US Large-Cap Core Responsible Index ETF,1.16%,--,--,--,--,--,--,04/03/23
19,CVMC,Calvert US Mid-Cap Core Responsible Index ETF,-4.15%,--,--,--,--,--,--,04/03/23
20,CVIE,Calvert International Responsible Index ETF,1.78%,--,--,--,--,--,--,04/03/23
21,CDEI,"Calvert US Large-Cap Diversity, Equity and Inc...",2.91%,--,--,--,--,--,--,04/03/23


In [58]:
driver.quit()

## Construindo a Tabela Final

In [59]:
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.set_index('Ticker')
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs[['1 Year','3 Years','5 Years']]

tabela_cadastro_etfs = tabela_cadastro_etfs.set_index('Ticker')

In [60]:
tabela_rentabilidade_etfs

,1 Year,3 Years,5 Years
Ticker,,,
SPY,-7.72%,20.17%,11.41%
IVV,-7.75%,20.20%,11.44%
VOO,-7.73%,20.23%,11.45%
VTI,-8.92%,20.23%,10.64%
QQQ,-10.85%,21.20%,16.09%
...,...,...,...
CVLC,--,--,--
CVMC,--,--,--
CVIE,--,--,--


In [61]:
base_de_dados_final = tabela_cadastro_etfs.join(tabela_rentabilidade_etfs, how ='inner')
base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$371.98B,-7.72%,20.17%,11.41%
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$305.66B,-7.75%,20.20%,11.44%
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$285.90B,-7.73%,20.23%,11.45%
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$281.86B,-8.92%,20.23%,10.64%
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$172.39B,-10.85%,21.20%,16.09%
...,...,...,...,...,...,...,...,...
CVLC,Calvert US Large-Cap Core Responsible Index ETF,Equity: U.S. - Large Cap,Morgan Stanley,0.15%,--,--,--,--
CVMC,Calvert US Mid-Cap Core Responsible Index ETF,Equity: U.S. - Mid Cap,Morgan Stanley,0.15%,--,--,--,--
CVIE,Calvert International Responsible Index ETF,Equity: Developed Markets Ex-U.S. - Large Cap,Morgan Stanley,0.18%,--,--,--,--
